In [14]:
import numpy as np
import pandas as pd

In [15]:
ticker = 'AAPL'
df_sim = pd.read_csv(f"../price_sim/{ticker}_sim.csv")
df_sim

,pred_delta_ratio,real_delta_ratio,pred_delta,real_delta,real_price
0,0.426718,-0.090699,1.345067,-0.08,133.41
1,0.214654,0.449669,0.780239,1.35,134.76
2,0.175662,0.385430,0.671739,1.18,135.94
3,-0.082796,-0.336320,0.028877,-0.73,135.21
4,0.682156,-0.037795,1.991047,0.06,135.27
...,...,...,...,...,...
447,-1.250595,-1.987654,-3.000688,-5.10,230.76
448,-0.622708,-0.132265,-1.411231,-0.19,230.57
449,0.774646,0.256951,2.245893,0.84,231.41
450,0.635950,0.691512,1.825455,1.99,233.40


In [16]:
np.corrcoef(df_sim['pred_delta_ratio'], df_sim['real_delta_ratio'])

array([[1.        , 0.26731525],
       [0.26731525, 1.        ]])

In [17]:
np.corrcoef(df_sim['pred_delta'], df_sim['real_delta'])

array([[1.       , 0.2618984],
       [0.2618984, 1.       ]])

In [18]:
def sigmoid(x, k=1):
    return np.array(1 / (1 + np.exp(-k * x)))

In [19]:
def arctan(x, k=1):
  return np.array(np.arctan(k * x) / (np.pi / 4))

In [20]:
def sharpe_ratio(capital, risk_free=0.0):
    returns = np.diff(capital) / capital[:-1]
    excess = np.mean(returns - risk_free)
    std_dev = np.std(returns)
    return (excess / std_dev) * np.sqrt(252)

In [21]:
def trade(ratios, prices):
    capital = []
    currency = [1.0]
    position = [0.0]
    for ratio, price in zip(ratios[1:], prices[:-1]):
        money = currency[-1] + position[-1] * price
        assert money >= 0, "Bankruptcy"
        capital.append(money)
        nxt_cur = money * (1 - ratio)
        nxt_pos = money * ratio / price
        currency.append(nxt_cur)
        position.append(nxt_pos)
    money = currency[-1] + position[-1] * prices[-1]
    capital.append(money)

    return pd.DataFrame({
        'ratios': ratios,
        'prices': prices,
        'capital': capital,
        'currency': currency,
        'position': position
    })

In [22]:
annual_risk_free = 0.04
daily_risk_free = (1+annual_risk_free) ** (1/252) - 1
daily_risk_free

0.0001556498627912628

In [23]:
ratios1, prices1 = np.ones(len(df_sim), dtype=float), df_sim['real_price'].to_numpy()
df_trade1 = trade(ratios1, prices1)
df_trade1['prices'].iloc[-1] / df_trade1['prices'].iloc[0], df_trade1['capital'].iloc[-1]

(1.7515178772205982, 1.7515178772206033)

In [24]:
ratios5, prices5 = sigmoid(df_sim['pred_delta'], 5), df_sim['real_price'].to_numpy()
df_trade5 = trade(ratios5, prices5)
df_trade5['prices'].iloc[-1] / df_trade5['prices'].iloc[0], df_trade5['capital'].iloc[-1]

(1.7515178772205982, 2.692128384557236)

In [25]:
ratios6, prices6 = arctan(df_sim['pred_delta'], 1), df_sim['real_price'].to_numpy()
df_trade6 = trade(ratios6, prices6)
df_trade6['prices'].iloc[-1] / df_trade6['prices'].iloc[0], df_trade6['capital'].iloc[-1]

(1.7515178772205982, 6.177542870619419)

In [26]:
print(f"Always hold stock: (Sharpe){sharpe_ratio(df_trade1['capital'], daily_risk_free):.2f}, (Yield){df_trade1['capital'].iloc[-1]:.2f}")
print(f"Our strategy (no short selling): (Sharpe){sharpe_ratio(df_trade5['capital'], daily_risk_free):.2f}, (Yield){df_trade5['capital'].iloc[-1]:.2f}")
print(f"Our strategy (short selling): (Sharpe){sharpe_ratio(df_trade6['capital'], daily_risk_free):.2f}, (Yield){df_trade6['capital'].iloc[-1]:.2f}")

Always hold stock: (Sharpe)1.37, (Yield)1.75
Our strategy (no short selling): (Sharpe)3.32, (Yield)2.69
Our strategy (short selling): (Sharpe)4.02, (Yield)6.18
